In [1]:
import pandas as pd
import numpy as np

In [6]:
df1 = pd.read_csv('dataset1.txt',header=None, sep="\t")
df2 = pd.read_csv('dataset2.txt',header=None, sep="\t")
df3 = pd.read_csv('dataset3.txt',header=None, sep="\t")
df4 = pd.read_csv('dataset4.txt',header=None, sep="\t")
df5 = pd.read_csv('dataset5.txt',header=None, sep="\t")

In [7]:
df = pd.concat([df1,df2,df3,df4,df5], axis=0, ignore_index=True)

In [8]:
df = df.drop_duplicates(subset=[0])

In [9]:
SEQUENCE_LENGTH = 200

In [10]:
def max_x_chars(max_len):
    def truncate_sentence(str):
        splitted_str = str.split(" ")
        word_lengths = [len(w) for w in splitted_str]
        cum_sum = np.cumsum(word_lengths)
        end = [i for i,x in enumerate(cum_sum) if x <= max_len][-1]
        return ' '.join(splitted_str[:end+1])
    return truncate_sentence

truncate = max_x_chars(SEQUENCE_LENGTH)

In [11]:
df = df[df[0].apply(lambda o: type(o) is str)]
len(df)

5784637

In [12]:
df[0] = df[0].apply(truncate)

In [13]:
df['original'] = df[0]

# expected output. 'hello world' become 'h e l l o # w o r l d'
df['expected'] = df[0].apply(lambda x: ' ## '.join([' '.join(w) for w in x.split(' ')]))

# input. 'hello world' become 'h e l l o w o r l d'
df['sentence'] = df[0].apply(lambda x: ' '.join([' '.join(w) for w in x.split(' ')]))
print(len(df))
df.head()

5784637


,0,original,expected,sentence
0,NEW ORLEANS — Whenever a Virginia Tech offensi...,NEW ORLEANS — Whenever a Virginia Tech offensi...,N E W ## O R L E A N S ## — ## W h e n e v e r...,N E W O R L E A N S — W h e n e v e r a V i r ...
1,inevitably the first road game in 2008 against...,inevitably the first road game in 2008 against...,i n e v i t a b l y ## t h e ## f i r s t ## r...,i n e v i t a b l y t h e f i r s t r o a d g ...
2,Midway through the first quarter,Midway through the first quarter,M i d w a y ## t h r o u g h ## t h e ## f i r...,M i d w a y t h r o u g h t h e f i r s t q u ...
3,Virginia Tech had to call two timeouts in a ro...,Virginia Tech had to call two timeouts in a ro...,V i r g i n i a ## T e c h ## h a d ## t o ## ...,V i r g i n i a T e c h h a d t o c a l l t w ...
4,and,and,a n d,a n d


In [14]:
df = df.sample(frac=1.0)
pair_list = list(zip(df['sentence'].values, df['expected'].values))

train_length = int(0.8 * len(pair_list))
eval_test_length = int(0.1 * len(pair_list))

tmp = pair_list
#train set
train_l = tmp[:train_length]

tmp = tmp[train_length:]
#validation set
eval_l = tmp[:eval_test_length]

tmp = tmp[eval_test_length:]
#test set
test_l = tmp[:eval_test_length]

In [15]:
#list of string saved to file
def list_to_file(list, filename):
    with open(filename, 'w') as fp:
        for el in list:
            fp.write(f"{el}\n")

#split (train-test,validation) saved to files
def split_to_files(split, name):
    list_to_file([s for s,_ in split], f"./{name}-source.txt")
    list_to_file([t for _,t in split], f"./{name}-target.txt")

In [16]:
splits = {
    'train' : train_l,
    'test' : test_l,
    'eval' : eval_l
}
for k,v in splits.items():
    split_to_files(v,k)